In [1]:
%cd ../src

/home/ubuntu/SPVD_Lightning/src


In [ ]:
from my_models.spvd import SPVUnet

down_blocks = [{
    "features": [32, 64, 128, 192, 192, 256],
    "num_layers": 1,
    "attn_heads": (None, None, None, 8, 8)
}]
up_blocks = [
    {
        "features": [256, 192, 192],
        "num_layers": 1,
        "attn_heads": 8,
    },
    {
        "features": [192, 128, 64, 32],
        "num_layers": 1,
        "attn_heads": None
    },
]
t_emb_features = 64

model = SPVUnet(down_blocks, up_blocks, t_emb_features)

import models

model = models.DiffusionBase(model, lr=1e-4)

In [ ]:
import torch

model_state_dict = torch.load("../checkpoints/ModelNet40/airplane-cross.ckpt", weights_only=True)
model.load_state_dict(model_state_dict)
model = model.cuda().eval()

In [ ]:
from my_schedulers.ddpm_scheduler import DDPMSparseScheduler

ddpm_sched = DDPMSparseScheduler(beta_min=0.0001, beta_max=0.02, steps=1024)

In [ ]:
preds = ddpm_sched.sample(model, 16, 2048)

In [ ]:
from utils.visualization import visualize_notebook
visualize_notebook(preds, x_offset=7, y_offset=7, point_size=0.2)